In [2]:
import os
import libsql
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

url = os.getenv("CLAUDELIGHT_DB_URL")
auth_token = os.getenv("CLAUDELIGHT_RW")
conn = libsql.connect("claude_light.db", sync_url=url, auth_token=auth_token)
conn.sync()

In [ ]:
map_weather = {}
map_cloud_cover = {}

def convert_time():
    dfd
    return []
    
def convert_weather_Pi():
    return 
    
def P_cloud_cover():
    return

In [16]:
# Read data from db and covert to df

df = pd.DataFrame()
# for rowid, data in conn.execute("""select * from measurements where json_extract(data, '$.tag') = '20250731'""").fetchall():
for rowid, data in conn.execute("""select * from measurements where json_extract(data, '$.tag') IS NOT NULL""").fetchall():
    j = json.loads(data)
    
    R, G, B = j['in']
    selected_outputs = ['445nm', '515nm', '630nm', 'clear']
    out_values = [j['out'][key] for key in selected_outputs]

    flat_data = {
        'R': R,
        'G': G,
        'B': B,
        'I_445': out_values[0],
        'I_515': out_values[1],
        'I_630': out_values[2],
        'I_clear': out_values[3]
    }
    df_oneRow = pd.DataFrame([flat_data])
    
    df = pd.concat([df, df_oneRow], ignore_index=True)

print(df.shape)
df.head(3)

(6, 7)


,R,G,B,I_445,I_515,I_630,I_clear
0,0.0,0.0,0.0,844,1689,2127,13266
1,0.9,0.5,0.6,19888,35221,30901,65535
2,1.0,0.8,0.2,7964,54316,34207,65535


In [4]:
# Split train, val, test
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

df_X_train = df_train[['R', 'G', 'B']]
df_Y_train = df_train[['I_445', 'I_515', 'I_630', 'I_clear']]
df_X_test = df_test[['R', 'G', 'B']]
df_Y_test = df_test[['I_445', 'I_515', 'I_630', 'I_clear']]

df_X_train.head(3)

,R,G,B
4,0.1,0.8,0.4
2,0.8,1.0,0.6
0,0.9,0.5,0.6


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# Train MultiOutput Random Forest
base_model = RandomForestRegressor(n_estimators=100, random_state=42)
multi_rf = MultiOutputRegressor(base_model)
multi_rf.fit(df_X_train, df_Y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions using test data
Y_train_pred = multi_rf.predict(df_X_train)
df_Y_train_pred = pd.DataFrame(Y_train_pred, columns=['I_445_pred', 'I_515_pred', 'I_630_pred', 'I_clear_pred'])

# Evaluate
list_metric_rows = []
for i, target in enumerate(['I_445', 'I_515', 'I_630', 'I_clear']):
    mae = mean_absolute_error(df_Y_train.iloc[:, i], df_Y_train_pred.iloc[:, i])
    mse = mean_squared_error(df_Y_train.iloc[:, i], df_Y_train_pred.iloc[:, i])
    r2 = r2_score(df_Y_train.iloc[:, i], df_Y_train_pred.iloc[:, i])
    list_metric_rows.append({
        'target': target,
        'MAE': mae,
        'MSE': mse,
        'R2': r2
    })

df_metrics_train = pd.DataFrame(list_metric_rows)
df_metrics_train

,target,MAE,MSE,R2
0,I_445,1838.9250,5.209558e+06,0.810452
1,I_515,7226.1450,6.715807e+07,0.849383
2,I_630,3344.0725,1.348073e+07,0.907683
3,I_clear,1355.7350,3.544003e+06,0.864000


In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions using test data
Y_pred = multi_rf.predict(df_X_test)
df_Y_pred = pd.DataFrame(Y_pred, columns=['I_445_pred', 'I_515_pred', 'I_630_pred', 'I_clear_pred'])

# Evaluate
list_metric_rows = []
for i, target in enumerate(['I_445', 'I_515', 'I_630', 'I_clear']):
    mae = mean_absolute_error(df_Y_test.iloc[:, i], df_Y_pred.iloc[:, i])
    mse = mean_squared_error(df_Y_test.iloc[:, i], df_Y_pred.iloc[:, i])
    r2 = r2_score(df_Y_test.iloc[:, i], df_Y_pred.iloc[:, i])
    list_metric_rows.append({
        'target': target,
        'MAE': mae,
        'MSE': mse,
        'R2': r2
    })

df_metrics = pd.DataFrame(list_metric_rows)
df_metrics

c:\Users\26070\anaconda3\envs\torch\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\26070\anaconda3\envs\torch\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\26070\anaconda3\envs\torch\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\26070\anaconda3\envs\torch\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


,target,MAE,MSE,R2
0,I_445,9837.71,9.678054e+07,NaN
1,I_515,1713.13,2.934814e+06,NaN
2,I_630,9915.02,9.830762e+07,NaN
3,I_clear,117.89,1.389805e+04,NaN
